In [2]:
import pickle
class Foo:
    attr='A class attribute'
picklesting=pickle.dumps(Foo)

In [4]:
def save(obj):
    return (obj.__class__, obj.__dict__)
def load(cls,attributes):
    obj = cls.__new__(cls)
    obj.__dict__update(attributes)
    return obj

In [3]:
import pickle
import sqlite3
from collections import namedtuple

MemoRecord = namedtuple("MemoRecord","key, task")

class DBPickler(pickle.Pickler):
    def persisten_id(self,obj):
        if isinstance(obj,MemoRecord):
            return("Memorecord",obj.key)
        else:
            return None
class DBUnpickler(pickle.Unpickler):
    def __init__(self,file,connection):
        super().__init__(file)
        self.connection = connection
    def persistent_load(self,pid):
        cursor = self.connection.cursor()
        type_tag, key_id = pid 
        if type_tag == "MemoRecord":
            cursor.execute("SELECT * FROM memos WHERE key= ? ", (str(key_id),))
            key, task = cursor.fetchone()
            return MemoRecord(key, task)
        else:
            raise pickle.UnpicklingError("unsupported persisten object")
def main():
    import io
    import pprint
    conn=sqlite3.connect(":memory:")
    cursor=conn.cursor()
    cursor.execute("CREATE TABLE memos(key INTEGER PRIMARY KEY , task TEXT)")
    tasks = (
    'give food to fish',
    'prepare group meeting',
    'fight with a zebra',
    )
    for task in tasks:
        cursor.execute("INSERT INTO memos VALUES(NULL, ?)",(task,))
    cursor.execute("SELECT * FROM memos")
    memos = [MemoRecord(key, task) for key, task in cursor]
    file = io.BytesIO()
    DBPickler(file).dump(memos)
    print("Pickled records:")
    pprint.pprint(memos)
    cursor.execute("UPDATE memos SET task='learn italian' WHERE key=1")
    file.seek(0)
    memos = DBUnpickler(file, conn).load()
    print("Unpickled records:")
    pprint.pprint(memos)
    
if __name__ == "__main__":
    main()
    

Pickled records:
[MemoRecord(key=1, task='give food to fish'),
 MemoRecord(key=2, task='prepare group meeting'),
 MemoRecord(key=3, task='fight with a zebra')]
Unpickled records:
[MemoRecord(key=1, task='give food to fish'),
 MemoRecord(key=2, task='prepare group meeting'),
 MemoRecord(key=3, task='fight with a zebra')]


In [4]:
class TextReader:
    def __init__(self,filename):
        self.filename = filename
        self.file=open(filename)
        self.lineno=0
    def readline(self):
        self.lineno += 1
        line = self.file.readline()
        if not line:
            return None
        if line.endswith('\n'):
            line=line[:-1]
        return "%i: %s" %(self.lineno, line)
    def __getstate__(self):
        state=self.__dict__.copy()
        del state['file']
        return state
    def __setstate__(self,state):
        self.__dict__.update(state)
        file=open(self.filename)
        for _ in range(self.lineno):
            file.readline()
        self.file =file
            

In [6]:
reader=TextReader("hello.txt")
reader.readline()

'1: 1: Hello world!'

In [7]:
reader.readline()

'2: 2: I am line number two.'

In [8]:
new_reader=pickle.loads(pickle.dumps(reader))

In [9]:
new_reader.readline()

'3: 3: Goodbye!'

In [11]:
import io
import pickle
class MyClass:
    my_attribute = 1
class MyPickler(pickle.Pickler):
    def reducer_override(self, obj):
        if getattr(obj, "__name__", None) == "MyClass":
            return type, (obj.__name__, obj.__bases__, {'my_attribute': obj.my_attribute})
        else:
            return NotImplemented
f = io.BytesIO()
p = MyPickler(f)
p.dump(MyClass)

del MyClass

unpickled_class = pickle.loads(f.getvalue())

assert isinstance(unpickled_class, type)
assert unpickled_class.__name__ == "MyClass"
assert unpickled_class.my_attribute == 1

AttributeError: Can't get attribute 'MyClass' on <module '__main__'>

In [12]:
class ZeroCopyByteArray(bytearray):
    def __reduce_ex__(self, protocol):
        if protocol >= 5:
            return type(self)._reconstruct, (PickleBuffer(self),),None
        else:
            return type(self)._reconstruct, (bytearray(self),)
    @classmethod
    def _reconstruct(cls, obj):
        with memoryview(obj) as m:
            obj = m.obj
            if type(obj) is cls:
                return obj
            else:
                return cls(obj)

In [14]:
b = ZeroCopyByteArray(b"abc")
data = pickle.dumps(b, protocol=3)
new_b = pickle.loads(data)
print(b == new_b)
print(b is new_b)

True
False


In [18]:
import pickle
pickle.loads(b"cos\nsystem\n(S'echo hello world '\ntR.")

0

In [19]:
import builtins
import io
import pickle
safe_builtins = {
    'range',
    'complex',
    'set',
    'frozenset',
    'slice',
}

class RestrictedUnpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == "builtins" and name in safe_builtins:
            return getattr(builtins, name)
        raise pickle.UnpicklingError("global '%s.%s' is forbidden" %
                                    (module, name))
def restricted_loads(s):
    return RestrictedUnpickler(io.BytesIO(s)).load()

In [20]:
restricted_loads(pickle.dumps([1,2, range(15)]))

[1, 2, range(0, 15)]

In [21]:
restricted_loads(b"cos\nsystem\n(S'echo hello world'\ntR.")

UnpicklingError: global 'os.system' is forbidden

In [25]:
restricted_loads(b'cbuiltins\neval\n'
                 b'(S\'getattr(__import__("os"),"system")'
                 b'("echo hello world")\'\ntR.')

UnpicklingError: global 'builtins.eval' is forbidden

In [26]:
import  pickle
data = {
    'a' : [1,2.0,3,4+6j],
    'b' : ("character string",b"byte string"),
    'c' : {None, True, False}
}

with open('data.pickle','wb') as f:
    pickle.dump(data, f, pickle.HIGHEST_PROTOCOL)

In [27]:
import pickle
with open('data.pickle','rb') as f:
    data=pickle.load(f)

In [28]:
data

{'a': [1, 2.0, 3, (4+6j)],
 'b': ('character string', b'byte string'),
 'c': {False, None, True}}

In [3]:
import copyreg, copy, pickle
class C(object):
    def __init__(self, a):
        self.a = a 
def pickle_c(c):
        print("pickling a C instance...")
        return C, (c.a,)

In [4]:
copyreg.pickle(C, pickle_c)

In [5]:
c = C(1)
d = copy.copy(c)
p = pickle.dumps(c)

pickling a C instance...
pickling a C instance...


In [7]:
import shelve
with shelve.open('spam') as db:
    db['eggs'] = 'eggs'

In [9]:
import dbm 
with dbm.open('cache','c') as db:
    db[b'hello'] = b'there'
    db['www.python.org'] = 'Python website'
    db['www.cnn.com'] = 'Cable News Network'
    assert db[b'www.python.org'] == b'Python Website'
    assert db['www.cnn.com'] == b'Cable News Network'
    print(db.get('python.org', b'not present'))
    db['www.yahoo.com']=4

AssertionError: 

In [10]:
import sqlite3 
conn = sqlite3.connect('example.db')

In [11]:
c = conn.cursor()
c.execute(''' CREATE TABLE stocks(date text , trans text , symbol text , qty real , price real)''')
c.execute(" INSERT INTO stocks VALUES('2006-01-05','BUY','RHAT',100,35.14)")
conn.commit()
conn.close()

In [12]:
import sqlite3
conn=sqlite3.connect('example.db')
c=conn.cursor()
t=('RHAT',)
c.execute('SELECT * FROM stocks WHERE symbol=?',t)
print(c.fetchone())

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)


In [13]:
purchases = [('2006-03-28', 'BUY', 'IBM', 1000, 45.00),
             ('2006-04-05', 'BUY', 'MSFT', 1000, 72.00),
             ('2006-04-06', 'SELL', 'IBM', 500, 53.00),
            ]
c.executemany('INSERT INTO stocks values(?,?,?,?,?)', purchases)

In [15]:
for row in c.execute('SELECT * FROM stocks ORDER BY price'):
    print(row)

('2006-01-05', 'BUY', 'RHAT', 100.0, 35.14)
('2006-03-28', 'BUY', 'IBM', 1000.0, 45.0)
('2006-04-06', 'SELL', 'IBM', 500.0, 53.0)
('2006-04-05', 'BUY', 'MSFT', 1000.0, 72.0)


In [1]:
import sqlite3 
con = sqlite3.connect(":memory:")
con.isolation_level=None
cur=con.cursor()
buffer=""
print("Enter your sql command to execute in sqlite")
print("Enter a blank line to exit")
while True:
    line = input()
    if line == "":
        break
    buffer += line
    if sqlite3.complete_statement(buffer):
        try:
            buffer= buffer.strip()
            cur.execute(buffer)
            if buffer.lstrip().upper().startswith("SELECT"):
                print(cur.fetchall())
        except sqlite3.Error as e:
                print("An error occurred",e.args[0])
        buffer =""
con.close()

Enter your sql command to execute in sqlite
Enter a blank line to exit


 SELECT * FROM DUAL
 


In [2]:
import sqlite3
import hashlib
def md5sum(t):
    return hashlib.md5(t).hexdigest()
con=sqlite3.connect(":memory:")
con.create_function("md5",1,md5sum)
cur=con.cursor()
cur.execute("select md5(?)",(b"foo",))
print(cur.fetchone()[0])
con.close()

acbd18db4cc2f85cedef654fccc4a4d8


In [3]:
import sqlite3
class MySum:
    def __init__(self):
        self.count=0
    def step(self,value):
        self.count += value
    def finalize(self):
        return self.count
con = sqlite3.connect(":memory:")
con.create_aggregate("mysum",1,MySum)
cur=con.cursor()
cur.execute("create table test(i)")
cur.execute("insert into test(i) values(1)")
cur.execute("insert into test(i) values(2)")
cur.execute("select mysum(i) from test")
print(cur.fetchone()[0])
con.close()

3


In [4]:
import sqlite3
def collate_reverse(string1,string2):
    if string1 == string2:
        return 0
    elif string1 < string2 :
        return 1
    else:
        return -1
con = sqlite3.connect(":memory:")
con.create_collation("reverse",collate_reverse)
cur=con.cursor()
cur.execute("create table test(x)")
cur.executemany("insert into test(x) values(?)",[("a",),("b",)])
cur.execute("select x from test order by x collate reverse")
for row in cur:
    print(row)
con.close()

('b',)
('a',)


In [5]:
import sqlite3
con = sqlite3.connect(":memory:")
con.enable_load_extension(True)
con.execute("select load_extension('./fts3.so')")
con.enable_load_extension(False)
con.execute("create virtual table recipe using fts3(name, ingredients)")
con.executescript(""" 
    insert into recipe (name, ingredients) values ('broccoli stew', 'broccoli peppers cheese tomatoes');
    insert into recipe (name, ingredients) values ('pumpkin stew', 'pumpkin onions garlic celery');
    insert into recipe (name, ingredients) values ('broccoli pie', 'broccoli cheese onions flour');
    insert into recipe (name, ingredients) values ('pumpkin pie', 'pumpkin sugar flour butter');
""")
for row in con.execute("select rowid , name , ingredients from recipe where name match 'pie'"):
    print(row)

OperationalError: The specified module could not be found.
